# Sesión 8 - Policy Gradients: Implementación de PG, bPG, y AC

> En la presente sesión, se va a desarrollar la **implementación**  algoritmos dentro de la familia de **Policy Gradients (PG)** En concreto, se va a implementar **Vanilla Policy Gradients (PG)**, **baseline - Policy Gradients (bPG)**, y **Actor-Critic (AC)**. La implementación será realizada utilizando la librería **pytorch**, librería de más bajo nivel, y la cuál permitirá trabajar en detalle la estrategia de aprendizaje on-policy y la optimización por gradiente de la policy.


---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.



---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda update --all
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```




---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [ ]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/VIU/08_AR_MIAR/sesiones_practicas/sesion_practica_2"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [ ]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

---
### 1.4. Instalar librerías necesarias


In [ ]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.5.3
  %pip install torch==2.0.1
  %pip install agents==1.4.0

---
### 1.5.Acerca de las librerías para RL

Librería para trabajar con nuestros entornos: gym (https://gym.openai.com/) \
Librería para trabajar con deep learning: tensorflow (https://www.tensorflow.org/) \
Librería para desarrollar soluciones de RL a alto nivel: keras-rl (https://github.com/keras-rl/keras-rl) \


---
## **PARTE 2** - *Implementación de algoritmos Policy Gradients*


---
### 2.1. Vanilla PG (PG)

In [1]:
# Imports
import math
import os
import sys
import time

import numpy as np
import gym
import torch

from PIL import Image
from torch.autograd import Variable

In [67]:
# Seleccionar device CPU/GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device: " + str(device))

# Variables
model_path = "breackout_pg.pth"
env_name = "BreakoutDeterministic-v4"

# Hyperparametros - Entrenamiento
EPISODES_TRAINING = 200
EPISODES_TESTING = 10
GAMMA = 0.99
T = 512
BS = 16

# Hyperparametros - Preprocesado observacion-stado
HEIGHT, WIDTH = 84, 84
N_FRAMES = 4

Device: cpu


In [3]:
# Definir arquitectura del knowledge (CNN)
class Actor(torch.nn.Module):
  def __init__(self, number_actions=4):
    super(Actor, self).__init__()

    # Definimos las capas de la red neuronal
    # 1) Base Model
    self.conv1 = torch.nn.Conv2d(4, 32, 8, stride=4)
    self.conv2 = torch.nn.Conv2d(32, 64, 4, stride=2)
    self.conv3 = torch.nn.Conv2d(64, 64, 3, stride=1)

    # 2) Proyección intermedia (Asumimos que hemos hecho flatten)
    self.fc1 = torch.nn.Linear(7*7*64, 512) # Para más generalizable, podria usarse GAP

    # 3) Salida
    self.actor = torch.nn.Linear(512, number_actions) # number of actions

  def forward(self, x):

    # 1) Forward base model
    x = torch.nn.functional.relu(self.conv1(x))
    x = torch.nn.functional.relu(self.conv2(x))
    x = torch.nn.functional.relu(self.conv3(x))

    # 2) Flatten + Projection
    x = x.view(-1, 7*7*64)
    x = torch.nn.functional.relu(self.fc1(x))

    # 3) Salida hibrida
    policy = self.actor(x) # Aplicaremos softmax más adelante

    return policy

def np_to_tensor(x):
    return torch.tensor(x).to(torch.float32)

In [4]:
## Funciones para pre-procesado

# Pasar de rgb a nivel de gris y re-escalado
def rgb2gray_and_resize(screen, height, width):

  # RGB a gris
  screen_gray = np.array(np.dot(screen[...,:3], [0.299, 0.587, 0.114]), dtype=np.uint8)
  # pixel (1,1) -> r=10, g=20, b=30 -> gray = 10 * 0.299 + 20*0.587 + ...
  img_from_array = Image.fromarray(screen_gray)

  # Re-escalado de imagen
  img_from_array = img_from_array.resize((height, width))

  return np.array(img_from_array)

# Concatenar secuencias (window_lenght)
def update_frame_sequence(state, obs, n_frames=4, width=84, height=84):

  # Paso a nivel de gris, re-escalado, y estandarización
  obs = np.ascontiguousarray(rgb2gray_and_resize(obs, height, width), dtype=np.float32) / 255
  obs = torch.FloatTensor(obs)

  # Incorporar a buffer
  if state is None: # Inicio, no tenemos ventanas previas - repetimos obs inicial
    _state = obs.repeat(n_frames, 1).view(n_frames, width, height)
  else: # Tenemos ventanas previas, incorporamos la última ventana
    _state = state.view(n_frames, height, width)
    _state = torch.cat((_state[1:], obs.view(1, width, height)))

  return _state

In [40]:
# Entrenamiento de agente PG:
torch.manual_seed(123) # Reproducibilidad

# Instanciamos un entorno
env = gym.make(env_name)
env.seed(123)
number_actions = env.action_space.n

# Instanciamos un entorno
env = gym.make(env_name)
number_actions = env.action_space.n

# Instanciamos el modelo del actor
model = Actor(number_actions=number_actions).to(device)
model.eval() # en train, tendriamos que hacer: model.train()

# Preparar optimizador
optimizer = torch.optim.Adam(model.parameters(), lr=0.00025)

# Inicializamos primera trayectoria
obs, state = env.reset(), None
state = update_frame_sequence(state=state, obs=obs, width=WIDTH, height=HEIGHT)

# Primer bucle: episodios de entrenamiento
for i_episode in range(int(EPISODES_TRAINING)):
  done = False

  # ------------------------------------------------------------------------------------
  # A. Recopilar trayectoria

  # Buffer de memoria (s,a,r,s)
  s, a, r, lives = [], [], [], [] # Lives is enviroment-specific information

  # Segundo bucle: recopilamos la trayectoria
  for step in range(int(T)):

    # Foward de actor dado el estado actual
    with torch.no_grad():
        logits = model(state.unsqueeze(0).to(device))

    # Obtenemos probabilidad de acción
    prob = torch.nn.functional.softmax(logits, -1) # [0, 0.2, 0.6, 0.2]

    # Obtener la acción a realizar
    action = prob.multinomial(num_samples=1) # 2

    # Con la accion seleccionada, realizamos un step en el entorno
    obs, reward, done, info = env.step(action.item())

    # Almacenamos información en memoria
    a_ohe = torch.nn.functional.one_hot(action, num_classes=number_actions).squeeze(0) # Actions to ohe
    s.append(state.unsqueeze(0).numpy()), a.append(a_ohe.numpy()), r.append(reward), lives.append(info['ale.lives'])
    
    # Actualizamos el estado con la siguiente observacion
    state = update_frame_sequence(state=state, obs=obs, width=WIDTH, height=HEIGHT)
    
    if done:
      obs, state = env.reset(), None
      state = update_frame_sequence(state=state, obs=obs, width=WIDTH, height=HEIGHT)
      break
    
  # Printear evolución
  print("Trayectoria " + str(int(i_episode)+1) + "/" + str(int(EPISODES_TRAINING)))
  print("Numero de steps en la trayectoria: " + str(len(r)) + " -- Recompensa total del episodio: " + str(np.sum(r)))

  # ------------------------------------------------------------------------------------
  # B. Preparar la información

  # Computar los discount rewards
  running_r = 0
  discount_rewards = r.copy()
  for i in reversed(range(len(r))):
    if lives[i] != lives[i-1]:
        running_r = 0
    # Obtenemos discount rewards para cada t
    running_r = r[i] + GAMMA * running_r
    discount_rewards[i] = running_r

  # ------------------------------------------------------------------------------------
  # C. Entrenar policy con trayectoria recolectada
  model.train()
  BS = 16

  idx, track_loss = 0, 0
  indexes = np.arange(0, len(s))
  np.random.shuffle(indexes)
  for i_epoch in range(len(s)//BS):
      indexes_batch = indexes[idx:idx+BS]

      # Seleccionamos un batch de la trayectoria
      states_batch = np_to_tensor(np.concatenate(s)[indexes_batch,:,:,:])
      actions_batch = np_to_tensor(np.concatenate(a)[indexes_batch,:].squeeze())
      relevance_batch = np_to_tensor(np.array(discount_rewards)[indexes_batch])
    
      # Hacemos forward al actor-critic dado el estado actual
      logits = model(states_batch)

      # Obtenemos las probabilidades de la acción a partir de los logits
      log_softmax = torch.nn.functional.log_softmax(logits, -1)

      # Obtenemos criterios de optimización
      policy_loss = - torch.mean((log_softmax * actions_batch.detach()).sum(-1) * relevance_batch.detach())

      # Computamos gradientes
      policy_loss.backward()
      # Actualizamos los pesos
      optimizer.step()
      # Limpiamos gradientes del modelo
      optimizer.zero_grad()
      # Actualizamos iterador de batch
      idx += BS
      track_loss += policy_loss.item()/(len(s)//BS)
      
      # Track losses
      print(str(i_epoch) + "/" + str(len(s)//BS) + " - loss: " + str(policy_loss.item()), end="\r")
        
  # Overall loss
  print(str(i_epoch) + "/" + str(len(s)//BS) + " - loss: " + str(track_loss), end="\n")
 
  # Guardar los pesos del modelo
  torch.save(model.state_dict(), model_path)

  # Set again model to eval
  model.eval()

# Guardar los pesos del modelo
torch.save(model.state_dict(), "last" + model_path)


Trayectoria 1/200
Numero de steps en la trayectoria: 168 -- Recompensa total del episodio: 1.0
9/10 - loss: 0.28632478117942817
Trayectoria 2/200
Numero de steps en la trayectoria: 138 -- Recompensa total del episodio: 0.0
7/8 - loss: 0.00
Trayectoria 3/200
Numero de steps en la trayectoria: 144 -- Recompensa total del episodio: 0.0
8/9 - loss: 0.00
Trayectoria 4/200
Numero de steps en la trayectoria: 211 -- Recompensa total del episodio: 2.0
12/13 - loss: 0.5727335122915415
Trayectoria 5/200
Numero de steps en la trayectoria: 132 -- Recompensa total del episodio: 0.0
7/8 - loss: 0.00
Trayectoria 6/200
Numero de steps en la trayectoria: 172 -- Recompensa total del episodio: 1.0
9/10 - loss: 0.28001164346933366
Trayectoria 7/200
Numero de steps en la trayectoria: 174 -- Recompensa total del episodio: 1.0
9/10 - loss: 0.25513680800795554
Trayectoria 8/200
Numero de steps en la trayectoria: 180 -- Recompensa total del episodio: 1.0
10/11 - loss: 0.24941720136187295
Trayectoria 9/200
Numer

Trayectoria 68/200
Numero de steps en la trayectoria: 150 -- Recompensa total del episodio: 1.0
8/9 - loss: 0.24858109487427604
Trayectoria 69/200
Numero de steps en la trayectoria: 162 -- Recompensa total del episodio: 1.0
9/10 - loss: 0.25833650156855583
Trayectoria 70/200
Numero de steps en la trayectoria: 132 -- Recompensa total del episodio: 0.0
7/8 - loss: 0.00
Trayectoria 71/200
Numero de steps en la trayectoria: 171 -- Recompensa total del episodio: 1.0
9/10 - loss: 0.26960633546113966
Trayectoria 72/200
Numero de steps en la trayectoria: 174 -- Recompensa total del episodio: 1.0
9/10 - loss: 0.25806112512946134
Trayectoria 73/200
Numero de steps en la trayectoria: 141 -- Recompensa total del episodio: 0.0
7/8 - loss: 0.00
Trayectoria 74/200
Numero de steps en la trayectoria: 155 -- Recompensa total del episodio: 1.0
8/9 - loss: 0.26886878824896283
Trayectoria 75/200
Numero de steps en la trayectoria: 178 -- Recompensa total del episodio: 1.0
10/11 - loss: 0.23985476385463376
T

11/12 - loss: 0.25764200588067379
Trayectoria 136/200
Numero de steps en la trayectoria: 224 -- Recompensa total del episodio: 3.0
13/14 - loss: 0.5568393394351006
Trayectoria 137/200
Numero de steps en la trayectoria: 53 -- Recompensa total del episodio: 1.0
2/3 - loss: 0.47395081321398425
Trayectoria 138/200
Numero de steps en la trayectoria: 224 -- Recompensa total del episodio: 3.0
13/14 - loss: 0.5621513639177594
Trayectoria 139/200
Numero de steps en la trayectoria: 11 -- Recompensa total del episodio: 0.0
13/0 - loss: 0
Trayectoria 140/200
Numero de steps en la trayectoria: 224 -- Recompensa total del episodio: 3.0
13/14 - loss: 0.99513370650155254
Trayectoria 141/200
Numero de steps en la trayectoria: 43 -- Recompensa total del episodio: 0.0
1/2 - loss: 0.00
Trayectoria 142/200
Numero de steps en la trayectoria: 182 -- Recompensa total del episodio: 1.0
10/11 - loss: 0.26290538907051086
Trayectoria 143/200
Numero de steps en la trayectoria: 158 -- Recompensa total del episodio:

In [41]:
# Testeo
def test(env, model, runs):
    model.eval()

    for e in range(runs):
        obs, state = env.reset(), None
        state = update_frame_sequence(state=state, obs=obs, width=WIDTH, height=HEIGHT)
        done, score = False, 0
        while not done:
            env.render()
            
            # Al testear, seleccionamos la acción con mayor probabilidad (no muestreo)
            with torch.no_grad():
                logits = model(state.unsqueeze(0).to(device))
                # Obtenemos probabilidad de acción
                prob = torch.nn.functional.softmax(logits, -1)
                # Obtener la acción a realizar
                action = prob.multinomial(num_samples=1)
                
            # Con la accion seleccionada, realizamos un step en el entorno
            obs, reward, done, info = env.step(action.item())

            # Actualizamos el estado con la siguiente observacion
            state = update_frame_sequence(state=state, obs=obs, width=WIDTH, height=HEIGHT)
            
            score += reward
            if done:
                env.close()
                print("episode: {}/{}, score: {}".format(e, runs, score))
                break
    
# Inferencia en test
env = gym.make(env_name)
model.load_state_dict(torch.load(model_path))
test(env, model, EPISODES_TESTING)


episode: 0/10, score: 2.0
episode: 1/10, score: 2.0
episode: 2/10, score: 0.0
episode: 3/10, score: 0.0
episode: 4/10, score: 3.0
episode: 5/10, score: 1.0
episode: 6/10, score: 0.0
episode: 7/10, score: 1.0
episode: 8/10, score: 0.0
episode: 9/10, score: 2.0


---
### 2.2. Baseline Policy Gradients (bPG)


In [65]:
# Variables
model_path = "breackout_bpg.pth"

In [68]:
# Entrenamiento de agente bPG:
torch.manual_seed(123) # Reproducibilidad

# Instanciamos un entorno
env = gym.make(env_name)
env.seed(123)
number_actions = env.action_space.n

# Instanciamos el modelo del actor
model = Actor(number_actions=number_actions).to(device)
model.eval() # en train, tendriamos que hacer: model.train()

# Preparar optimizador
optimizer = torch.optim.Adam(model.parameters(), lr=0.00025)

# Inicializamos primera trayectoria
obs, state = env.reset(), None
state = update_frame_sequence(state=state, obs=obs, width=WIDTH, height=HEIGHT)

# Primer bucle: episodios de entrenamiento
for i_episode in range(int(EPISODES_TRAINING)):
  done = False

  # ------------------------------------------------------------------------------------
  # A. Recopilar trayectoria

  # Buffer de memoria (s,a,r,s)
  s, a, r, lives = [], [], [], [] # Lives is enviroment-specific information

  # Segundo bucle: recopilamos la trayectoria
  for step in range(int(T)):

    # Foward de actor dado el estado actual
    with torch.no_grad():
        logits = model(state.unsqueeze(0).to(device))

    # Obtenemos probabilidad de acción
    prob = torch.nn.functional.softmax(logits, -1) # [0, 0.2, 0.6, 0.2]

    # Obtener la acción a realizar
    action = prob.multinomial(num_samples=1) # 2

    # Con la accion seleccionada, realizamos un step en el entorno
    obs, reward, done, info = env.step(action.item())

    # Almacenamos información en memoria
    a_ohe = torch.nn.functional.one_hot(action, num_classes=number_actions).squeeze(0) # Actions to ohe
    s.append(state.unsqueeze(0).numpy()), a.append(a_ohe.numpy()), r.append(reward), lives.append(info['ale.lives'])
    
    # Actualizamos el estado con la siguiente observacion
    state = update_frame_sequence(state=state, obs=obs, width=WIDTH, height=HEIGHT)
    
    if done:
      obs, state = env.reset(), None
      state = update_frame_sequence(state=state, obs=obs, width=WIDTH, height=HEIGHT)
      break
    
  # Printear evolución
  print("Trayectoria " + str(int(i_episode)+1) + "/" + str(int(EPISODES_TRAINING)))
  print("Numero de steps en la trayectoria: " + str(len(r)) + " -- Recompensa total del episodio: " + str(np.sum(r)))

  # ------------------------------------------------------------------------------------
  # B. Preparar la información

  # Computar los discount rewards
  running_r = 0
  discount_rewards = r.copy()
  for i in reversed(range(len(r))):
    if lives[i] != lives[i-1]:
        running_r = 0
    # Obtenemos discount rewards para cada t
    running_r = r[i] + GAMMA * running_r
    discount_rewards[i] = running_r
    
  # Normalización para reducir la varianza
  discount_rewards = np.array(discount_rewards)
  discount_rewards -= np.mean(discount_rewards)
  discount_rewards /= (np.std(discount_rewards)+1e-3)
    
  # ------------------------------------------------------------------------------------
  # C. Entrenar policy con trayectoria recolectada
  model.train()
  BS = 16

  idx, track_loss = 0, 0
  indexes = np.arange(0, len(s))
  np.random.shuffle(indexes)
  for i_step in range(len(s)//BS):
      indexes_batch = indexes[idx:idx+BS]

      # Seleccionamos un batch de la trayectoria
      states_batch = np_to_tensor(np.concatenate(s)[indexes_batch,:,:,:])
      actions_batch = np_to_tensor(np.concatenate(a)[indexes_batch,:].squeeze())
      relevance_batch = np_to_tensor(discount_rewards[indexes_batch])
    
      # Hacemos forward al actor-critic dado el estado actual
      logits = model(states_batch)

      # Obtenemos las probabilidades de la acción a partir de los logits
      log_softmax = torch.nn.functional.log_softmax(logits, -1)

      # Obtenemos criterios de optimización
      policy_loss = - torch.mean((log_softmax * actions_batch.detach()).sum(-1) * relevance_batch.detach())
    
      # Computamos gradientes
      policy_loss.backward()
      # Actualizamos los pesos
      optimizer.step()
      # Limpiamos gradientes del modelo
      optimizer.zero_grad()
      # Actualizamos iterador de batch
      idx += BS
      track_loss += policy_loss.item()/(len(s)//BS)
      
      # Track losses
      print(str(i_step+1) + "/" + str(len(s)//BS) + " - loss: " + str(policy_loss.item()), end="\r")
        
  # Overall loss
  print(str(i_step+1) + "/" + str(len(s)//BS) + " - loss: " + str(track_loss), end="\n")
 
  # Guardar los pesos del modelo
  torch.save(model.state_dict(), model_path)

  # Set again model to eval
  model.eval()

# Guardar los pesos del modelo
torch.save(model.state_dict(), "last_" + model_path)

Trayectoria 1/200
Numero de steps en la trayectoria: 168 -- Recompensa total del episodio: 1.0
10/10 - loss: 0.002677291631698611
Trayectoria 2/200
Numero de steps en la trayectoria: 138 -- Recompensa total del episodio: 0.0
8/8 - loss: 0.00
Trayectoria 3/200
Numero de steps en la trayectoria: 144 -- Recompensa total del episodio: 0.0
9/9 - loss: 0.00
Trayectoria 4/200
Numero de steps en la trayectoria: 211 -- Recompensa total del episodio: 2.0
13/13 - loss: -0.007684062879819152
Trayectoria 5/200
Numero de steps en la trayectoria: 132 -- Recompensa total del episodio: 0.0
8/8 - loss: 0.00
Trayectoria 6/200
Numero de steps en la trayectoria: 214 -- Recompensa total del episodio: 3.0
13/13 - loss: -0.0040487675712658905
Trayectoria 7/200
Numero de steps en la trayectoria: 186 -- Recompensa total del episodio: 1.0
11/11 - loss: -0.03549315035343168
Trayectoria 8/200
Numero de steps en la trayectoria: 226 -- Recompensa total del episodio: 2.0
14/14 - loss: -0.0631856849150998228
Trayector

15/15 - loss: -0.043336061139901495
Trayectoria 65/200
Numero de steps en la trayectoria: 261 -- Recompensa total del episodio: 7.0
16/16 - loss: -0.013548823771998286
Trayectoria 66/200
Numero de steps en la trayectoria: 264 -- Recompensa total del episodio: 7.0
16/16 - loss: 0.0043425295734778056
Trayectoria 67/200
Numero de steps en la trayectoria: 290 -- Recompensa total del episodio: 11.0
18/18 - loss: 0.020772629727919892
Trayectoria 68/200
Numero de steps en la trayectoria: 288 -- Recompensa total del episodio: 7.0
18/18 - loss: -0.022449696643484954
Trayectoria 69/200
Numero de steps en la trayectoria: 300 -- Recompensa total del episodio: 8.0
18/18 - loss: 0.002161750776900187
Trayectoria 70/200
Numero de steps en la trayectoria: 310 -- Recompensa total del episodio: 11.0
19/19 - loss: -0.01881230504889236
Trayectoria 71/200
Numero de steps en la trayectoria: 282 -- Recompensa total del episodio: 7.0
17/17 - loss: -0.010475788922870863
Trayectoria 72/200
Numero de steps en la 

17/17 - loss: 0.022991910795955094
Trayectoria 127/200
Numero de steps en la trayectoria: 274 -- Recompensa total del episodio: 7.0
17/17 - loss: -0.001164321513736962
Trayectoria 128/200
Numero de steps en la trayectoria: 298 -- Recompensa total del episodio: 11.0
18/18 - loss: 0.044536889427238046
Trayectoria 129/200
Numero de steps en la trayectoria: 257 -- Recompensa total del episodio: 7.0
16/16 - loss: -0.0017901044338941574
Trayectoria 130/200
Numero de steps en la trayectoria: 249 -- Recompensa total del episodio: 7.0
15/15 - loss: -0.016345014174779255
Trayectoria 131/200
Numero de steps en la trayectoria: 215 -- Recompensa total del episodio: 3.0
13/13 - loss: -0.03338145550626975
Trayectoria 132/200
Numero de steps en la trayectoria: 251 -- Recompensa total del episodio: 7.0
15/15 - loss: -0.014179625610510498
Trayectoria 133/200
Numero de steps en la trayectoria: 256 -- Recompensa total del episodio: 7.0
16/16 - loss: -0.009930118918418884
Trayectoria 134/200
Numero de step

17/17 - loss: 0.0339474235387409456
Trayectoria 189/200
Numero de steps en la trayectoria: 289 -- Recompensa total del episodio: 11.0
18/18 - loss: 0.0196195617318153382
Trayectoria 190/200
Numero de steps en la trayectoria: 216 -- Recompensa total del episodio: 3.0
13/13 - loss: -0.012860997938192799
Trayectoria 191/200
Numero de steps en la trayectoria: 311 -- Recompensa total del episodio: 8.0
19/19 - loss: 0.02652416025337420505
Trayectoria 192/200
Numero de steps en la trayectoria: 259 -- Recompensa total del episodio: 7.0
16/16 - loss: -0.04008766042534262
Trayectoria 193/200
Numero de steps en la trayectoria: 275 -- Recompensa total del episodio: 11.0
17/17 - loss: -0.021222049151273344
Trayectoria 194/200
Numero de steps en la trayectoria: 247 -- Recompensa total del episodio: 7.0
15/15 - loss: 0.030032347018520045
Trayectoria 195/200
Numero de steps en la trayectoria: 257 -- Recompensa total del episodio: 7.0
16/16 - loss: -0.025922424159944057
Trayectoria 196/200
Numero de st

In [39]:
# Inferencia en test
env = gym.make(env_name)
model.load_state_dict(torch.load("last_" + model_path))
test(env, model, EPISODES_TESTING)

episode: 0/10, score: 10.0
episode: 1/10, score: 11.0
episode: 2/10, score: 4.0
episode: 3/10, score: 7.0
episode: 4/10, score: 7.0
episode: 5/10, score: 11.0
episode: 6/10, score: 4.0
episode: 7/10, score: 11.0
episode: 8/10, score: 7.0
episode: 9/10, score: 7.0


---
### 2.3. Actor-Critic (AC)

In [ ]:
# Variables
model_path = "breackout_ac.pth"

In [42]:
# Definir arquitectura del knowledge (CNN)
class ActorCritic(torch.nn.Module):
  def __init__(self, number_actions=4):
    super(ActorCritic, self).__init__()

    # Definimos las capas de la red neuronal
    # 1) Base Model
    self.conv1 = torch.nn.Conv2d(4, 32, 8, stride=4)
    self.conv2 = torch.nn.Conv2d(32, 64, 4, stride=2)
    self.conv3 = torch.nn.Conv2d(64, 64, 3, stride=1)

    # 2) Proyección intermedia (Asumimos que hemos hecho flatten)
    self.fc1 = torch.nn.Linear(7*7*64, 512) # Para más generalizable, podria usarse GAP

    # 3) Salida hibrida
    self.actor = torch.nn.Linear(512, number_actions) # number of actions
    self.critic = torch.nn.Linear(512, 1) # linear output of value

  def forward(self, x):

    # 1) Forward base model
    x = torch.nn.functional.relu(self.conv1(x))
    x = torch.nn.functional.relu(self.conv2(x))
    x = torch.nn.functional.relu(self.conv3(x))

    # 2) Flatten + Projection
    x = x.view(-1, 7*7*64)
    x = torch.nn.functional.relu(self.fc1(x))

    # 3) Salida hibrida
    policy = self.actor(x) # Aplicaremos softmax más adelante
    value = self.critic(x)

    return policy, value

In [ ]:
# Entrenamiento de agente Actor-Critic:
torch.manual_seed(123) # Reproducibilidad

# Instanciamos un entorno
env = gym.make(env_name)
env.seed(123)
number_actions = env.action_space.n

# Instanciamos el modelo del actor
model = ActorCritic(number_actions=number_actions).to(device)
model.eval() # en train, tendriamos que hacer: model.train()

# Preparar optimizador
optimizer = torch.optim.Adam(model.parameters(), lr=0.00025)

# Inicializamos primera trayectoria
obs, state = env.reset(), None
state = update_frame_sequence(state=state, obs=obs, width=WIDTH, height=HEIGHT)

# Primer bucle: episodios de entrenamiento
for i_episode in range(int(EPISODES_TRAINING)):
  done = False

  # ------------------------------------------------------------------------------------
  # A. Recopilar trayectoria

  # Buffer de memoria (s,a,r,s)
  s, a, r, lives, values = [], [], [], [], [] # Lives is enviroment-specific information

  # Segundo bucle: recopilamos la trayectoria
  for step in range(int(T)):

    # Foward de actor dado el estado actual
    with torch.no_grad():
        logits, v = model(state.unsqueeze(0).to(device))

    # Obtenemos probabilidad de acción
    prob = torch.nn.functional.softmax(logits, -1) # [0, 0.2, 0.6, 0.2]

    # Obtener la acción a realizar
    action = prob.multinomial(num_samples=1) # 2

    # Con la accion seleccionada, realizamos un step en el entorno
    obs, reward, done, info = env.step(action.item())

    # Almacenamos información en memoria
    a_ohe = torch.nn.functional.one_hot(action, num_classes=number_actions).squeeze(0) # Actions to ohe
    s.append(state.unsqueeze(0).numpy()), a.append(a_ohe.numpy()), r.append(reward), lives.append(info['ale.lives'])
    values.append(v.item())
    
    # Actualizamos el estado con la siguiente observacion
    state = update_frame_sequence(state=state, obs=obs, width=WIDTH, height=HEIGHT)
    
    if done:
      obs, state = env.reset(), None
      state = update_frame_sequence(state=state, obs=obs, width=WIDTH, height=HEIGHT)
      break
    
  # Printear evolución
  print("Trayectoria " + str(int(i_episode)+1) + "/" + str(int(EPISODES_TRAINING)))
  print("Numero de steps en la trayectoria: " + str(len(r)) + " -- Recompensa total del episodio: " + str(np.sum(r)) + 
        " -- Value promedio: " + str(np.mean(values)))

  # ------------------------------------------------------------------------------------
  # B. Preparar la información
    
  # Recompensa en estado final (terminal o no terminal)
  running_r = 0
  if not done:
    _, value = model(state.unsqueeze(0).to(device))
    running_r = v.item()
  
  # Computar los discount rewards
  discount_rewards = r.copy()
  for i in reversed(range(len(r))):
    if lives[i] != lives[i-1]:
        running_r = 0
    # Obtenemos discount rewards para cada t
    running_r = r[i] + GAMMA * running_r
    discount_rewards[i] = running_r
    
  # Normalización para reducir la varianza
  discount_rewards = np.array(discount_rewards)
  discount_rewards -= np.mean(discount_rewards)
  discount_rewards /= (np.std(discount_rewards)+1e-3)
    
  # ------------------------------------------------------------------------------------
  # C. Entrenar actor y critic
    
  # C.1. Entrenamos actor utilizando la trayectoria recolectada
  model.train()
  BS = 16

  idx, track_loss = 0, 0
  indexes = np.arange(0, len(s))
  np.random.shuffle(indexes)
  for i_step in range(len(s)//BS):
      indexes_batch = indexes[idx:idx+BS]

      # Seleccionamos un batch de la trayectoria
      states_batch = np_to_tensor(np.concatenate(s)[indexes_batch,:,:,:])
      actions_batch = np_to_tensor(np.concatenate(a)[indexes_batch,:].squeeze())
      relevance_batch = np_to_tensor(np.array(values)[indexes_batch])
    
      # Hacemos forward al actor-critic dado el estado actual
      logits, _ = model(states_batch)

      # Obtenemos las probabilidades de la acción a partir de los logits
      log_softmax = torch.nn.functional.log_softmax(logits, -1)

      # Obtenemos criterios de optimización
      policy_loss = - torch.mean((log_softmax * actions_batch.detach()).sum(-1) * relevance_batch.detach())

      # Computamos gradientes
      policy_loss.backward()
      # Actualizamos los pesos
      optimizer.step()
      # Limpiamos gradientes del modelo
      optimizer.zero_grad()
      # Actualizamos iterador de batch
      idx += BS
      track_loss += policy_loss.item()/(len(s)//BS)
      
      # Track losses
      print(str(i_step+1) + "/" + str(len(s)//BS) + " - policy loss: " + str(policy_loss.item()), end="\r")

  # C.2. Entrenamos el critic con las discount rewards
  model.train()
  BS = 16

  idx, track_loss_value = 0, 0
  indexes = np.arange(0, len(s))
  np.random.shuffle(indexes)
  for i_step in range(len(s)//BS):
      indexes_batch = indexes[idx:idx+BS]

      # Seleccionamos un batch de la trayectoria
      states_batch = np_to_tensor(np.concatenate(s)[indexes_batch,:,:,:])
      target_value = np_to_tensor(discount_rewards[indexes_batch])
    
      # Hacemos forward al actor-critic dado el estado actual
      _, values = model(states_batch)

      # Obtenemos criterios de optimización
      value_loss = torch.mean((target_value - values).pow(2))
    
      # Computamos gradientes
      value_loss.backward()
      # Actualizamos los pesos
      optimizer.step()
      # Limpiamos gradientes del modelo
      optimizer.zero_grad()
      # Actualizamos iterador de batch
      idx += BS
      track_loss_value += value_loss.item()/(len(s)//BS)
      
      # Track losses
      print(str(i_step+1) + "/" + str(len(s)//BS) + " - value loss: " + str(value_loss.item()), end="\r")
    
  # Overall loss
  print(str(i_step+1) + "/" + str(len(s)//BS) + " - policy loss: " + str(track_loss) +
        " - value loss: " + str(track_loss_value), end="\n")
 
  # Guardar los pesos del modelo
  torch.save(model.state_dict(), model_path)

  # Set again model to eval
  model.eval()

# Guardar los pesos del modelo
torch.save(model.state_dict(), "last_" + model_path)

In [ ]:
# Testeo
def test(env, model, runs):
    model.eval()

    for e in range(runs):
        obs, state = env.reset(), None
        state = update_frame_sequence(state=state, obs=obs, width=WIDTH, height=HEIGHT)
        done, score = False, 0
        while not done:
            env.render()
            
            # Al testear, seleccionamos la acción con mayor probabilidad (no muestreo)
            with torch.no_grad():
                logits, _ = model(state.unsqueeze(0).to(device))
                # Obtenemos probabilidad de acción
                prob = torch.nn.functional.softmax(logits, -1)
                # Obtener la acción a realizar
                action = prob.multinomial(num_samples=1)
                
            # Con la accion seleccionada, realizamos un step en el entorno
            obs, reward, done, info = env.step(action.item())

            # Actualizamos el estado con la siguiente observacion
            state = update_frame_sequence(state=state, obs=obs, width=WIDTH, height=HEIGHT)
            
            score += reward
            if done:
                env.close()
                print("episode: {}/{}, score: {}".format(e, runs, score))
                break
    
# Inferencia en test
env = gym.make(env_name)
model.load_state_dict(torch.load("last_" + model_path))
test(env, model, EPISODES_TESTING)

---
### 2.3. Advantadge Actor-Critic (A2C)
(sin multiproceso)

In [49]:
# Variables
model_path = "breackout_a2c_sp.pth"

In [ ]:
# Entrenamiento de agente A2C (single process):
torch.manual_seed(123) # Reproducibilidad

# Instanciamos un entorno
#env = gym.make(env_name)
env = gym.make(env_name, repeat_action_probability=0.0, frameskip=(1,2))
env.seed(123)
number_actions = env.action_space.n

# Instanciamos el modelo del actor
model = ActorCritic(number_actions=number_actions).to(device)
model.eval() # en train, tendriamos que hacer: model.train()

# Preparar optimizador
optimizer = torch.optim.Adam(model.parameters(), lr=0.00025)

# Inicializamos primera trayectoria
obs, state = env.reset(), None
state = update_frame_sequence(state=state, obs=obs, width=WIDTH, height=HEIGHT)

# Primer bucle: episodios de entrenamiento
for i_episode in range(int(EPISODES_TRAINING)):
  done = False

  # ------------------------------------------------------------------------------------
  # A. Recopilar trayectoria

  # Buffer de memoria (s,a,r,s)
  s, a, r, lives, values = [], [], [], [], [] # Lives is enviroment-specific information

  # Segundo bucle: recopilamos la trayectoria
  for step in range(int(T)):

    # Foward de actor dado el estado actual
    with torch.no_grad():
        logits, v = model(state.unsqueeze(0).to(device))

    # Obtenemos probabilidad de acción
    prob = torch.nn.functional.softmax(logits, -1) # [0, 0.2, 0.6, 0.2]

    # Obtener la acción a realizar
    action = prob.multinomial(num_samples=1) # 2

    # Con la accion seleccionada, realizamos un step en el entorno
    obs, reward, done, info = env.step(action.item())

    # Almacenamos información en memoria
    a_ohe = torch.nn.functional.one_hot(action, num_classes=number_actions).squeeze(0) # Actions to ohe
    s.append(state.unsqueeze(0).numpy()), a.append(a_ohe.numpy()), r.append(reward), lives.append(info['ale.lives'])
    values.append(v.item())
    
    # Actualizamos el estado con la siguiente observacion
    state = update_frame_sequence(state=state, obs=obs, width=WIDTH, height=HEIGHT)
    
    if done:
      obs, state = env.reset(), None
      state = update_frame_sequence(state=state, obs=obs, width=WIDTH, height=HEIGHT)
      break
    
  # Printear evolución
  print("Trayectoria " + str(int(i_episode)+1) + "/" + str(int(EPISODES_TRAINING)))
  print("Numero de steps en la trayectoria: " + str(len(r)) + " -- Recompensa total del episodio: " + str(np.sum(r)) + 
        " -- Value promedio: " + str(np.mean(values)))

  # ------------------------------------------------------------------------------------
  # B. Preparar la información
    
  # Recompensa en estado final (terminal o no terminal)
  running_r = 0
  if not done:
    _, value = model(state.unsqueeze(0).to(device))
    running_r = v.item()
  
  # Computar los discount rewards
  discount_rewards = r.copy()
  for i in reversed(range(len(r))):
    if lives[i] != lives[i-1]:
        running_r = 0
    # Obtenemos discount rewards para cada t
    running_r = r[i] + GAMMA * running_r
    discount_rewards[i] = running_r
    
  # Normalización para reducir la varianza
  discount_rewards = np.array(discount_rewards)
  discount_rewards -= np.mean(discount_rewards)
  discount_rewards /= (np.std(discount_rewards)+1e-3)
  
  # Calculate advantadge
  advantadge = discount_rewards - np.array(values)
    
  # ------------------------------------------------------------------------------------
  # C. Entrenar actor y critic
    
  # C.1. Entrenamos actor utilizando la trayectoria recolectada
  model.train()
  BS = 16

  idx, track_loss = 0, 0
  indexes = np.arange(0, len(s))
  np.random.shuffle(indexes)
  for i_step in range(len(s)//BS):
      indexes_batch = indexes[idx:idx+BS]

      # Seleccionamos un batch de la trayectoria
      states_batch = np_to_tensor(np.concatenate(s)[indexes_batch,:,:,:])
      actions_batch = np_to_tensor(np.concatenate(a)[indexes_batch,:].squeeze())
      relevance_batch = np_to_tensor(advantadge[indexes_batch])
    
      # Hacemos forward al actor-critic dado el estado actual
      logits, _ = model(states_batch)

      # Obtenemos las probabilidades de la acción a partir de los logits
      log_softmax = torch.nn.functional.log_softmax(logits, -1)

      # Obtenemos criterios de optimización
      policy_loss = - torch.mean((log_softmax * actions_batch.detach()).sum(-1) * relevance_batch.detach())
        
      # Computamos gradientes
      policy_loss.backward()
      # Actualizamos los pesos
      #optimizer.step()
      # Limpiamos gradientes del modelo
      #optimizer.zero_grad()
      # Actualizamos iterador de batch
      idx += BS
      track_loss += policy_loss.item()/(len(s)//BS)
      
      # Track losses
      print(str(i_step+1) + "/" + str(len(s)//BS) + " - policy loss: " + str(policy_loss.item()), end="\r")

  # C.2. Entrenamos el critic con las discount rewards
  model.train()
  BS = 16

  idx, track_loss_value = 0, 0
  indexes = np.arange(0, len(s))
  np.random.shuffle(indexes)
  for i_step in range(len(s)//BS):
      indexes_batch = indexes[idx:idx+BS]

      # Seleccionamos un batch de la trayectoria
      states_batch = np_to_tensor(np.concatenate(s)[indexes_batch,:,:,:])
      target_value = np_to_tensor(discount_rewards[indexes_batch])
    
      # Hacemos forward al actor-critic dado el estado actual
      _, values = model(states_batch)

      # Obtenemos criterios de optimización
      value_loss = torch.mean((target_value - values).pow(2))
    
      # Computamos gradientes
      value_loss.backward()
      # Actualizamos los pesos
      #optimizer.step()
      # Limpiamos gradientes del modelo
      #optimizer.zero_grad()
      # Actualizamos iterador de batch
      idx += BS
      track_loss_value += value_loss.item()/(len(s)//BS)
      
      # Track losses
      print(str(i_step+1) + "/" + str(len(s)//BS) + " - value loss: " + str(value_loss.item()), end="\r")
    
  # Actualizamos los pesos
  optimizer.step()
  # Limpiamos gradientes del modelo
  optimizer.zero_grad() 
    
  # Overall loss
  print(str(i_step+1) + "/" + str(len(s)//BS) + " - policy loss: " + str(track_loss) +
        " - value loss: " + str(track_loss_value), end="\n")
 
  # Guardar los pesos del modelo
  torch.save(model.state_dict(), model_path)

  # Set again model to eval
  model.eval()

# Guardar los pesos del modelo
torch.save(model.state_dict(), "last_" + model_path)

In [ ]:
# Inferencia en test
env = gym.make(env_name, repeat_action_probability=0, frameskip=1)
model.load_state_dict(torch.load(model_path))
test(env, model, EPISODES_TESTING)